In [38]:
def update_list(input_list):
    """
    Generate a list of lists of length n from input_list
    """
    output_list = []
    for list1 in input_list:
        max1 = max(list1)
        output_list = output_list + [list1 + [i] for i in range(0,max1+2)]
    return output_list

def generate_list(n):
    """
    Generate a list of lists of length n
    """
    list1 = [[0]]
    for i in range(0,n-1):
        list1 = update_list(list1)
    return list1

def split_list(lst, lengths):
    sublists = []
    start = 0
    for length in lengths:
        # Take a slice of the original list with the specified length
        sublists.append(lst[start:start + length])
        start += length
    return sublists

In [39]:
from itertools import permutations

def generate_numbers(lengths):
    
    list_sum = sum(lengths)
    list_of_lists = generate_list(list_sum)
    list_of_lists = [split_list(list1, lengths) for list1 in list_of_lists]
    return list_of_lists    

def invert_permutation(permutation):
    return {value: index for index, value in enumerate(permutation)}


def generate_permutations(list1):
    max1 = max([max(list) for list in list1])
    fst = [list[0] for list in list1]
    perms = []
    for p in permutations(range(10), max1+1):
        if 0 in [p[i] for i in fst]:
            pass
        else:
            translated = [[p[i] for i in list] for list in list1]
            dict1 = invert_permutation(p)
            perms.append((translated, dict1))
    return perms

In [40]:
from collections import defaultdict

def categorize_by_f(long_list, f):
    result = defaultdict(list)
    for x in long_list:
        result[f(x)].append(x)
    
        
    return dict(result)
    
def to_result(perm):
    summands, mapping = perm
    sum1 = sum([int(''.join(map(str, numbers))) for numbers in summands])
    sum1 = list(map(int, str(sum1)))
    translated = []
    counter = 97
    new = {}
    for digit in sum1:
        if digit in mapping.keys():
            translated.append(mapping[digit])
        elif digit in new.keys():
            translated.append(new[digit])
        else:
            new[digit] = chr(counter)
            translated.append(chr(counter))
            counter += 1
    return ''.join(map(str, translated))

def select_permutations(perms):

    x = categorize_by_f(perms, to_result)
    z = [v[0] for k, v in x.items() if len(v) == 1]
    summands = [[int(''.join(map(str, numbers)))for numbers in y] for y, _ in z]
    summands = [x + [sum(x)] for x in summands]
    return summands


In [41]:
def get_alphametics(lengths):

    alphametics = [select_permutations(generate_permutations(perm)) for perm in generate_numbers(lengths)]
    alphametics = [item for sublist in alphametics for item in sublist]
    return alphametics

In [211]:
x = get_alphametics([4,2])
print(x)

[[9999, 99, 10098], [5555, 50, 5605], [9999, 90, 10089], [9999, 91, 10090], [9999, 92, 10091], [1111, 91, 1202], [8888, 98, 8986], [9999, 19, 10018], [9999, 89, 10088], [8888, 99, 8987], [9999, 11, 10010], [9999, 22, 10021], [5555, 61, 5616], [9999, 10, 10009], [9999, 12, 10011], [9999, 20, 10019], [5550, 55, 5605], [9990, 99, 10089], [9991, 99, 10090], [9992, 99, 10091], [5550, 50, 5600], [7774, 74, 7848], [8889, 89, 8978], [9990, 90, 10080], [9991, 91, 10082], [9994, 94, 10088], [9995, 95, 10090], [5550, 56, 5606], [5556, 50, 5606], [9990, 91, 10081], [9990, 98, 10088], [9991, 90, 10081], [9991, 97, 10088], [9991, 98, 10089], [9997, 91, 10088], [9998, 90, 10088], [9998, 91, 10089], [8889, 98, 8987], [9991, 19, 10010], [9992, 29, 10021], [7779, 99, 7878], [8889, 99, 8988], [9991, 11, 10002], [9992, 22, 10014], [9995, 55, 10050], [9991, 10, 10001], [9991, 18, 10009], [9992, 20, 10012], [9992, 21, 10013], [9992, 27, 10019], [9995, 54, 10049], [9998, 81, 10079], [5551, 65, 5616], [9990, 

In [45]:
print(len(x))

81


In [48]:
def fix_word(word):
    for char in word:
        if char in "' ?.!/;:&":
            word = word.replace(char,'')
    return word

def duplit(word_list):
    words = word_list.copy()
    for word in word_list:
        for i in range(len(word)):
            words.append(word[:i]+word[i]+word[i:])
    return words




In [254]:
with open('common_words.txt', 'r') as file:
    common_words = file.read().split('\n')
with open('wordlist.txt', 'r') as file:
    all_words = file.read().split('\n')

common_words = [fix_word(word.upper()) for word in common_words]
all_words = [fix_word(word.upper()) for word in all_words]
words = common_words + all_words
extras = [chr(97+i) for i in range(26)]
extras = [i+i+i for i in extras]           
words = words + extras

duplit_words = duplit(words)
duplit_words = list(filter(lambda x: len(x) < 7, duplit_words))
duplit_words = list(dict.fromkeys((duplit_words)))

with open('common_words_parsed.txt', 'w') as file:
    for word in common_words:
        file.write(word + '\n')
with open('words_parsed.txt', 'w') as file:
    for word in words:
        file.write(word + '\n')
with open('duplit_words_parsed.txt', 'w') as file:
    for word in duplit_words:
        file.write(word + '\n')


def decompose(x):
    values = list(dict.fromkeys((x)))
    return values, ''.join([str(values.index(v)) for v in x])
def compose(values, x):
    return ''.join([values[int(v)] for v in x])
def decompose2(x):
    return decompose(x)[1]

common_word_dict = categorize_by_f(common_words, decompose2)
normal_word_dict = categorize_by_f(words, decompose2)
duplit_dict = categorize_by_f(duplit_words, decompose2)


In [249]:
def find_letters(numbers, map, dict1):
    if len(numbers) == 0:
        return map, True

    encode = decompose2(numbers[0])

    if encode in dict1:
        for word in dict1[encode]:
            x = match(numbers[0], word, map)
            if x:
                y, perfect = find_letters(numbers[1:], x, dict1)
                if perfect:
                    return y, perfect
            
    z, perfect = find_letters(numbers[1:], map, dict1)

    return z, False
    
def match(number, word, map1):
    map = map1.copy()

    for i in range(len(number)):
        if number[i] in map:
            if map[number[i]] != word[i]:
                return False
        else:
            if word[i] in map.values():
                return False
            map[number[i]] = word[i]
    return map

def sort_no_list(numbers):
    return sorted(numbers, key = lambda x: len(word_dict[decompose2(x)]))
    

In [259]:
import random

def find_alphametic(numbers, dict1): 
    # Given a list of numbers return worded versions
    numbers = [str(i) for i in numbers]
    sorted = sort_no_list(numbers)
    map, perfect = find_letters(sorted, {}, dict1)
    remainder = [chr(i) for i in range(65, 91) if chr(i) not in map.values()]
    letters = [i for i in ''.join(sorted) if i not in map.keys()]
    p = list(permutations(remainder, len(letters)))
    px = p[random.randint(0, len(p)-1)]
    for i in range(len(letters)):
        map[letters[i]] = px[i]
    alpha = []
    for number in numbers:
        alpha.append(''.join([map[i] for i in number]))
    return alpha, perfect

def find_alphametic_full(numbers):
    c_find, c_perfect = find_alphametic(numbers, common_word_dict)
    if c_perfect:
        return c_find, parse_alphametic(c_find), 'common'
    n_find, n_perfect = find_alphametic(numbers, normal_word_dict)
    if n_perfect:
        return n_find, parse_alphametic(n_find), 'normal'
    d_find, d_perfect = find_alphametic(numbers, duplit_word_dict)
    return d_find, parse_alphametic(d_find), 'duplit'

def parse_alphametic(words):
    return ' + '.join(words[:-1]) + ' = ' + words[-1]

In [261]:
print(find_alphametic_full([5653, 97825, 103478]))


(['EVEN', 'FORCE', 'JUNIOR'], 'EVEN + FORCE = JUNIOR', 'common')


In [206]:
p = permutations(['r','e','w'],2)
print(list(p))

[('r', 'e'), ('r', 'w'), ('e', 'r'), ('e', 'w'), ('w', 'r'), ('w', 'e')]


In [189]:
find_letters(['1530129','129012'], {})

({'1': 'A', '2': 'N', '9': 'D', '0': 'E'}, False)